In [4]:
%load_ext autoreload
%autoreload 2

In [21]:
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Flatten
from copy import copy, deepcopy

In [158]:
# HELPERS
def process(data):
    if not os.path.exists(os.path.join("graffiti_data",data)):
        raise FileNotFoundError("No such data folder. Try 'train', 'test', or 'valid'.")
    files = sorted([file for file in \
                    os.listdir(os.path.join("graffiti_data",data))\
                    if file[-4:] == ".jpg"])
    X = [load_img(os.path.join("graffiti_data",data,im)) for im in files]
    X = [img_to_array(im) for im in X]
    X = [im.reshape((1, im.shape[0], im.shape[1], im.shape[2])) for im in X]
    X = np.array([preprocess_input(im) for im in X])
    classes = pd.read_csv(os.path.join('graffiti_data',data,'_classes.csv'))\
    .sort_values("filename")
    classes.insert(1,"pixvals",list(X))
    classes.reset_index(inplace=True)
    new_path = os.path.join("graffiti_data",f"{data}_data_full.csv")
    if os.path.exists(new_path):
        os.remove(new_path)
    classes.to_csv(new_path)
    return classes

In [151]:
train = process("train")
train.shape

(1821, 7)

In [152]:
train.head(1)

,index,filename,pixvals,masterpiece,nograffiti,tags,throwups
0,1361,08ACAE21-BB01-4D06-9BCC-DCD4B012A848_jpg.rf.b3...,"[[[-26.939003, -18.779, -3.6800003], [-27.9390...",0,0,1,0


In [166]:
x_train = np.array(list(thing for thing in train.values[:,2]))
y_train = train.values[:,-4:].astype(np.float32)

In [163]:
x_train.shape

(1821, 400, 400, 3)

In [168]:
# Load VGG16 without the top layer and with pooling
base_model = VGG16(include_top=False, input_shape=(400, 400, 3), pooling='max')

# Prevent training on the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Properly connect new layers
x = Flatten()(base_model.output)  # Ensure the base model's output connects here
x = Dense(1024, activation='relu')(x)  # Connect this layer to the output of the Flatten layer
output = Dense(4, activation='softmax')(x)  # Connect this Dense layer to the previous Dense layer

# Finalize the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer="Adam", loss="CategoricalCrossentropy", metrics=['accuracy'])

# Print the model structure to verify it is correct
model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 400, 400, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 400, 400, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 400, 400, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 200, 200, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 200, 200, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 200, 200, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 100, 100, 128)     0  

In [169]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
57/57 [==============================] - 410s 7s/step - loss: 24.0115 - accuracy: 0.6914
Epoch 2/5
57/57 [==============================] - 456s 8s/step - loss: 2.1114 - accuracy: 0.8885
Epoch 3/5
35/57 [=================>............] - ETA: 2:57 - loss: 1.1165 - accuracy: 0.9321


KeyboardInterrupt

